In [1]:
# HIDDEN
import pandas as pd
import numpy as np
import bqplot
import matplotlib.pyplot as plt
import geopandas
import contextily as ctx

In [2]:
# HIDDEN
data_gdf = geopandas.read_file('earthquake.geojson') #geojson file
data_gdf = data_gdf.dropna(subset=['geometry'])

In [3]:
# HIDDEN
import ipywidgets
from IPython.display import display

In [4]:
# HIDDEN
xrange = ipywidgets.FloatRangeSlider(min=-125, max=-65.07, value=[-125, -65.07], description='longitude: ')
yrange = ipywidgets.FloatRangeSlider(min=24.61, max=49.96, value=[24.61, 49.96], description='latitude: ')
range_box = ipywidgets.VBox([xrange, yrange])

In [5]:
# HIDDEN
filter_box = ipywidgets.BoundedFloatText(
    value=4.5,
    min=4.5,
    max=7.5,
    step=0.1,
    description='Magnitude:',
    disabled=False
)

In [6]:
# HIDDEN
selection_box = ipywidgets.RadioButtons(
    options=['Magnitude', 'Event Type', 'Magnitude Type'],
    value='Magnitude',
    description='Classifier:',
    disabled=False
)

In [7]:
# HIDDEN
titles=['Classifier', 'Magnitude Filter', 'Range Setter']
children=[selection_box, filter_box, range_box]
tab = ipywidgets.Tab()
tab.children = children
for i, item in enumerate(titles):
    tab.set_title(i, item)

# Data Viz-455  Final Project 
### <ul> <li>Author: Xinyu Huang</li> <br> <li>Theme: Earthquake Data</li> </ul>

## About the Data

This visulization helps users discovering a subset of the earthquake data derives from USGS (United States Geological Survey) website (www.usgs.gov).

The dataset performed in this project is a query of the earthquake part of USGS. Some APIs are provided for users to query the data with certain conditions (time, magnitude and region, etc.) on the [search page](https://earthquake.usgs.gov/earthquakes/search/). This dataset includes the data of the earthquakes over 4.5 magnitude from 1950 to today in the [conterminous U.S. region](https://en.wikipedia.org/wiki/Contiguous_United_States). 

There are overall 2543 records in this dataset. Each record has 22 columns, including the time, the magnitude, the position, the depth, the source and other information of each earthquake event. The detail explaination of each column is avaliable at [here](https://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php).

## Discover the Data

To discover the dataset, we want to take a peek at the most straightforward and important information, that is the time, location and the magnitude of the events. The location will be presented in another interactive dashboard, thus in the following visulization, I'll make a quick plot about how the number of earthquake events change with time.

In [8]:
# HIDDEN
# 1. Data
data_gdf['year'] = data_gdf['time'].apply(lambda x: pd.Timestamp(x, unit='ms').year) 

In [9]:
# HIDDEN
# 2. Scale
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.LinearScale()

# 3. Axis
x_ax = bqplot.Axis(scale=x_sc, label='Year', 
                   tick_rotate=90, tick_style = {'text-anchor': 'start'}, label_offset='40px')
y_ax = bqplot.Axis(scale=y_sc, label='Number of Earthquakes', orientation='vertical')

# 4. Mark
my_bar = bqplot.Bars(x=data_gdf.groupby('year')['year'].count().index,
                     y=data_gdf.groupby('year')['year'].count(),
                     display_legend=True,
                     labels=['All'],
                     scales={'x':x_sc, 'y':y_sc})


# Another Layer of Mark
severe_data = data_gdf.loc[data_gdf['mag']>=6]

severe_bar = bqplot.Bars(x=severe_data.groupby('year')['year'].count().index,
                         y=severe_data.groupby('year')['year'].count(),
                         colors=['red'],
                         display_legend=True,
                         labels=['Magnitude Over 6.0'],
                         name='Magnitude Over 6.0',
                         scales={'x':x_sc, 'y':y_sc})

# 4. Fig
fig = bqplot.Figure(marks=[my_bar, severe_bar], 
                    axes=[x_ax, y_ax],
                    legend_style={'stroke-width': 0}
                   )
display(fig)

Figure(axes=[Axis(label='Year', label_offset='40px', scale=OrdinalScale(), tick_rotate=90, tick_style={'text-a…

Magnitude reflects the physical size of an earthquake. Although some other factors should be taken into account, magnitude is still a very imporatant scale to estimate the severity of an earthquake event. From the visulization, we might able to search for rules of the period of the earthquake occurance. However, the time span of 70 years is still too small to study the law in the field of geology. Scientists need more data to conduct more insightful and more persuasive researches.

The strong earthquakes also have some research significance. Generally, we define the *strong* as the magnitude over 6.0. These earthquakes are also presented in the red bars.

## Dashboard

The following dashboard presents the location of each earthquake event. To make the plot fancier, this project introduced the contextily package, which is able to draw the map for us as long as we provide the earth coordinates of a specific area.

In [10]:
# HIDDEN
def my_plot(present_method, user_mag, xlim, ylim):
    subset_gdf = data_gdf.loc[data_gdf['mag']>=user_mag]
    
    fig, ax = plt.subplots(figsize=(15,5))
    
    ax.set_xlim(xlim[0], xlim[1])
    ax.set_ylim(ylim[0], ylim[1])
    
    if present_method == 'Magnitude':
        subset_gdf.plot(column='mag', cmap='Reds', legend=True, ax=ax, alpha=0.5, edgecolor='k')
    elif present_method == 'Event Type':
        subset_gdf.plot(column='type', cmap='rainbow', legend=True, ax=ax, alpha=0.5, edgecolor='k')
    elif present_method == 'Magnitude Type':
        subset_gdf.plot(column='magType', cmap='rainbow', legend=True, ax=ax, alpha=0.5, edgecolor='k')

    ctx.add_basemap(ax=ax, crs=data_gdf.crs)
    plt.show()


out = ipywidgets.interactive_output(my_plot, {'present_method': selection_box, 
                                           'user_mag': filter_box, 
                                           'xlim': xrange,
                                           'ylim': yrange})

display(out, tab)
# Can't see the tab in html, try to use an HBox instead

Output()

In [11]:
# HIDDEN
ipywidgets.HBox([selection_box, filter_box, range_box])

Please try discovering the dataset on the dashboard above! <br>
* You can change the way to classify the records by selecting other options by selecting in the **Classifier** box.
* You can set the lowest level of magnitude in the **Magnitude** box. The plot will show only the records whose magnitude is higher than the lowest level. 
* You can slide the **sliders** to set the range of latitude and longitude of shown in the plot.

** This dashboard is not smart enough. Each time you interact with the dashboard, the backend program need to draw the whole map again. Thus, it might take a while getting the feedback when you open this project or playing with the widgets. Wait patiently, please!

<br>

How's the earthquakes distributed? It's obvious that most of the earthquakes are concentrated in the west coast and looks like a line. We can easily guess that this part is on a seismic belt. It is acutally a part of the [Ring of Fire](https://en.wikipedia.org/wiki/Ring_of_Fire).

## Contextual Visualization

![World Earthquake Report for April 2021](https://volcanodiscovery.de/uploads/pics/quakes-4-2021.jpg)

Some online reports present the earthquake data in their specical way. Above is a monthly earthquake report published on [volcanodiscovery.com](https://www.volcanodiscovery.com/). It's about the world earthquake event happened in April, 2021. Similar to our dashboard, this visualization also presents the location data, but focuses on the data over the world. Thus, they applies a different projection, the equal earth projection for their map. 

Beside the location, this visualization also labeled the magnitude for each point and colored them according to their depth.
<br>
<br>

![title](Major_Earthquakes.png)

This is another interactive online visualization about earthquake data. It's also from [volcanodiscovery.com](https://www.volcanodiscovery.com/). <br>
You can get access to the sources page [here](https://www.volcanodiscovery.com/earthquakes/major.html) and try to play with it by yourself.

This visualization is based on a smarter tool--[leaflet](https://leafletjs.com/plugins.html). You can interact with the map more smoothly. It also provides a magnitude filter for the users. You can zoom in, zoom out the map and also see the details of the data point by hovering mouse on and click it. It's the ideal visualization this project want to achieve.

## Reference
<ol>
  <li>United States Geological Survey - <a href="https://earthquake.usgs.gov/earthquakes/search/">Search Earthquake Catalog</a></li>
  <li>Volcano Discovery - <a href="https://www.volcanodiscovery.com/earthquake/news.html">World Earthquake Report</a></li>
  <li>Volcano Discovery - <a href="https://www.volcanodiscovery.com/earthquakes/major.html">Big earthquakes</a></li>
  <li>Visualization - Nbinteract
</ol>